In [1]:
import numpy as np

class MultipleLinearRegression:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.learning_rate = learning_rate  # Learning rate for gradient descent
        self.iterations = iterations        # Number of iterations for training
        self.weights = None                 # Model parameters (including bias)
        self.mean = None                    # For feature scaling
        self.std = None                     # For feature scaling
        
    def _scale_features(self, X):
        """Standardize features (except the first column of ones)"""
        X_scaled = X.copy()
        if self.mean is not None and self.std is not None:
            # Skip first column (bias term)
            X_scaled[:, 1:] = (X[:, 1:] - self.mean) / self.std
        return X_scaled
    
    def fit(self, X, y):
        """Train the multiple linear regression model"""
        # Convert to numpy arrays and ensure proper shape
        X = np.array(X)
        y = np.array(y).reshape(-1)
        
        # Add bias term (column of 1s)
        X = np.c_[np.ones(X.shape[0]), X]
        
        # Store scaling parameters (mean/std) for each feature
        self.mean = np.mean(X[:, 1:], axis=0)
        self.std = np.std(X[:, 1:], axis=0)
        
        # Scale features
        X_scaled = self._scale_features(X)
        
        # Initialize weights
        n_features = X_scaled.shape[1]
        self.weights = np.zeros(n_features)
        
        # Gradient Descent
        for _ in range(self.iterations):
            # Calculate predictions
            y_pred = X_scaled @ self.weights
            
            # Calculate gradients
            error = y_pred - y
            gradients = (1 / len(y)) * X_scaled.T @ error
            
            # Update weights
            self.weights -= self.learning_rate * gradients
            
        return self
    
    def predict(self, X):
        """Make predictions using trained model"""
        X = np.array(X)
        
        # Add bias term
        X = np.c_[np.ones(X.shape[0]), X]
        
        # Scale features using training parameters
        X_scaled = self._scale_features(X)
        
        # Make prediction
        return X_scaled @ self.weights
    
    def score(self, X, y):
        """Calculate R-squared score"""
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)
    
    def mse(self, X, y):
        """Calculate Mean Squared Error"""
        y_pred = self.predict(X)
        return np.mean((y - y_pred) ** 2)


In [2]:
# Example usage
if __name__ == "__main__":
    # Generate sample data with 3 features
    np.random.seed(0)
    X = np.random.rand(100, 3)  # 100 samples, 3 features
    y = 4 + 3 * X[:, 0] + 2 * X[:, 1] - 5 * X[:, 2] + np.random.randn(100)  # True relationship with noise

    # Create and train model
    model = MultipleLinearRegression(learning_rate=0.01, iterations=5000)
    model.fit(X, y)

    # Make predictions
    X_test = np.array([[0.5, 0.5, 0.5], [1, 1, 1]])
    predictions = model.predict(X_test)

    # Print results
    print("Multiple Linear Regression Results")
    print("=================================")
    print(f"Weights (including bias): {model.weights}")
    print("\nTest predictions:")
    for i, pred in enumerate(predictions):
        print(f"Input {X_test[i]} -> Predicted y: {pred:.2f}")
    print(f"\nR-squared score: {model.score(X, y):.4f}")
    print(f"MSE: {model.mse(X, y):.4f}")


Multiple Linear Regression Results
Weights (including bias): [ 3.73457834  0.7681519   0.57822376 -1.32978465]

Test predictions:
Input [0.5 0.5 0.5] -> Predicted y: 3.86
Input [1. 1. 1.] -> Predicted y: 3.77

R-squared score: 0.7455
MSE: 0.8650
